# Setting runtime environment

In [ ]:
RUN_FROM = 'LOCAL'
# RUN_FROM = 'COLAB'

if RUN_FROM == 'LOCAL':
    from os.path import expanduser
    HOME = expanduser("~")
elif RUN_FROM == 'COLAB':
    !pip install hanziconv
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/My Drive/projects/lm
    HOME = '/content/drive/My Drive'

# Import modules

In [ ]:
import os
from data import Dataset
from model import Model
from routines import *
from utils import data_utils_py3
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

# Some presets

## CharVocabs

In [ ]:
CharVocabs = {
    'zh': {
        'vocab_file': HOME+"/Data/Vocab/char_vocab_zh",
        'embedding_files': HOME+"/Data/Vocab/zh_char_300_nlpcc.txt"
    },
    'en': {
        'vocab_file': HOME+"/Data/Vocab/char_vocab_en",
        'embedding_files': HOME+"/Data/Vocab/en_char_300_minimaxir.txt"
    }
}

## DataSets

In [ ]:
DataSets = {
    'text_zh': HOME+"/Data/text_zh",
    'nlpcc2017_news': HOME+"/Data/nlpcc2017_news",
    'smp2017_ecdt1': HOME+"/Data/smp2017_ecdt1",
}

## ModelConfigs

In [ ]:
ModelConfigs = {
    'small': {
        'char_embed_dim': 300,
        'layer_size': 256,
        'num_layers': 4,
        'num_heads': 8,
    },
    'medium': {
        'char_embed_dim': 300,
        'layer_size': 640,
        'num_layers': 10,
        'num_heads': 10,
    },
    'large': {
        'char_embed_dim': 300,
        'layer_size': 896,
        'num_layers': 16,
        'num_heads': 14,
    },
}

## TaskConfigs

In [ ]:
TaskConfigs = {
    'pretrain': {
        'task_spec': [
            {
                'type': 'sequence',
                'copy_from': [],
                'target_level': 0,
                'group_id': 0,
            },
        ],
    },
    'seq2cls': {
        'task_spec': [
            {
                'type': 'sequence',
                'copy_from': [],
                'target_level': 0,
                'group_id': 0,
            },
            {
                'type': 'class',
                'copy_from': [],
                'target_level': 1,
                'group_id': 1,
            },
        ],
    },
}

## Change run_config here

In [ ]:
run_config = {
    'task_config_name': 'pretrain',
    'char_vocab_name': 'zh',
    'model_config_name': 'small',
    'model_name': 'iter0',
#     'warm_start_from': {
#         'task_config_name': 'pretrain',
#         'model_name': 'iter0',
#         'vars_to_warm_start': '.*',
#     },
    'dataset_name': 'text_zh',
    'field_mapping': [0],
    'hyper_params': {
        'batch_size': 32,
        'max_train_steps': 100000,
        'max_lr': 5e-4,
        'pct_start': 0.3,
        'dropout': 0.1,
        'wd': 1e-6,
    },
}

char_vocab = data_utils_py3.AtomicVocab(
    filename=CharVocabs[run_config['char_vocab_name']]['vocab_file'],
    embedding_files=CharVocabs[run_config['char_vocab_name']]['embedding_files'])
dataset = Dataset(DataSets[run_config['dataset_name']], char_vocab)
def get_model_dir(*args):
    dir_name = '-'.join(args)
    if RUN_FROM == 'LOCAL':
        dir_path = dir_name
    elif RUN_FROM == 'COLAB':
        dir_path = os.path.join(HOME+'/Models/', dir_name)
    return dir_path
train_dir = get_model_dir(
    run_config['task_config_name'],
    run_config['char_vocab_name'],
    run_config['model_config_name'],
    run_config['model_name'])
warm_start_from = run_config.get('warm_start_from')
if warm_start_from is None:
    pretrain_dir = None
    vars_to_warm_start = None
else:
    pretrain_dir = get_model_dir(
        warm_start_from['task_config_name'],
        run_config['char_vocab_name'],
        run_config['model_config_name'],
        warm_start_from['model_name'])
    vars_to_warm_start = warm_start_from['vars_to_warm_start']
model = Model(
    ModelConfigs[run_config['model_config_name']],
    TaskConfigs[run_config['task_config_name']],
    char_vocab,
    train_dir,
    pretrain_dir=pretrain_dir,
    vars_to_warm_start=vars_to_warm_start)

# Let's Begin!

## LR range test and plot the curve

In [ ]:
lr_range_test(
    dataset, model.unfreeze(),
    run_config['field_mapping'], run_config['hyper_params'])

## Adjust hyper_params

In [ ]:
run_config['hyper_params'].update(
    {
        'max_lr': 5e-4,
    })

## Start train and eval loop

In [ ]:
train_and_evaluate(
    dataset, model.unfreeze(),
    run_config['field_mapping'], run_config['hyper_params'],
    eval_every=int(run_config['hyper_params']['max_train_steps']/4),
    distributed=True)

## Evaluate

In [ ]:
evaluate(
    dataset, model,
    run_config['field_mapping'], run_config['hyper_params'])

## Predict

In [ ]:
predict(dataset, model,
    run_config['field_mapping'], run_config['hyper_params'])